In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/machine-predictive-maintenance-classification/predictive_maintenance.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.columns[:2]

In [ ]:
df.drop(df.columns[:2] , axis = 1, inplace = True)

In [ ]:
cat_cols = df.select_dtypes(include = 'O').columns.tolist()
cat_cols

In [ ]:
for col in cat_cols :
    print(df[col].value_counts())

In [ ]:
df_f = df[df.Target == 1]
df_f['Failure Type'].value_counts()

In [ ]:
df_f[df_f['Failure Type'] == 'No Failure']

In [ ]:
df_f[df_f['Failure Type'] == 'No Failure'].index

In [ ]:
idx = df_f[df_f['Failure Type'] == 'No Failure'].index

In [ ]:
df.drop(idx , axis =0 , inplace = True )

In [ ]:
df_f = df[df.Target == 0]
df_f['Failure Type'].value_counts()

In [ ]:
idx = df_f[df_f['Failure Type'] == 'Random Failures'].index
df.drop(idx , axis =0 , inplace = True )

In [ ]:
df.info()

In [ ]:
num_cols = df.select_dtypes(exclude = 'O').columns.tolist()
num_cols

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
for i , col in enumerate(num_cols,1):
    print(i , col)

In [ ]:
plt.figure(figsize = (15 , 10))
for i , col in enumerate(num_cols,1):
    plt.subplot(6,1,i)
    sns.histplot(data = df , x= col)

In [ ]:
plt.figure(figsize = (15 , 10))
for i , col in enumerate(num_cols,1):
    plt.subplot(2,3,i)
    sns.histplot(data = df , x= col)

In [ ]:
plt.figure(figsize = (15 , 10))
for i , col in enumerate(num_cols,1):
    plt.subplot(2,3,i)
    plt.hist(data = df , x= col)

In [ ]:
sns.pairplot(df)

In [ ]:
sns.pairplot(df , hue = 'Target')

In [ ]:
df.head()

In [ ]:
df.columns[-2:]

In [ ]:
x = df.drop(df.columns[-2:] , axis = 1 )
y = df['Target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
cat_cols = X_train.select_dtypes(include = 'O').columns.tolist()
cat_cols

In [ ]:
num_cols = X_train.select_dtypes(exclude = 'O').columns.tolist()
num_cols

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
enc = OneHotEncoder(handle_unknown='ignore')
sc = StandardScaler()

ct = ColumnTransformer(
    [
        ('encoding' , enc , cat_cols ) ,
        ('scaling' , sc , num_cols)
    ]
)

In [ ]:
x_train = ct.fit_transform(X_train)
x_test = ct.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train , y_train)
clf.score(x_test , y_test)

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train , y_train)
svc.score(x_test , y_test)

In [ ]:
from sklearn.svm import SVC
rsvc = SVC(kernel = 'rbf')
rsvc.fit(x_train , y_train)
rsvc.score(x_test , y_test)

In [ ]:
y.value_counts()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(x_train, y_train)

In [ ]:
from collections import Counter
print(sorted(Counter(y_resampled).items()))

In [ ]:
clf.fit(X_resampled , y_resampled)

In [ ]:
clf.score(X_resampled , y_resampled)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(x_train, y_train)

In [ ]:
print(sorted(Counter(y_resampled).items()))

In [ ]:
clf.fit(X_resampled , y_resampled)

In [ ]:
clf.score(X_resampled , y_resampled)

In [ ]:
clf.score(x_test , y_test)

In [ ]:
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy =0.5)
und = RandomUnderSampler()
pipe = Pipeline(
    [
        ('o' , over) ,
        ('u' , und)
    ]
    )
x_s ,y_s = pipe.fit_resample(x_train , y_train)

In [ ]:
print(sorted(Counter(y_s).items()))

In [ ]:
clf.fit(x_s, y_s)

In [ ]:
clf.score(x_test , y_test)